### Install Pyspark

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 41.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f2686fb889fbe1813d68af554ef1a8228ef3daf58a57f3c972b749190245ab42
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


### Create Session
##### The entry point to programming Spark with the Dataset

In [2]:
from pyspark.sql import SparkSession

In [3]:
session = SparkSession.builder.appName("sql").master("local").getOrCreate()

In [5]:
data = session.read.csv("churn.csv", header = True, inferSchema = True)

In [7]:
data.show(5)

+----------+------+-------------+-------+----------+------+-----------+-------------------+------------------+-------------------+-------------------+-----------------------+-------------------+-------------------+-------------------+--------------+-------------+----------------+---------------------+-----------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|CallService|MultipleConnections|InternetConnection|     OnlineSecurity|       OnlineBackup|DeviceProtectionService|      TechnicalHelp|           OnlineTV|       OnlineMovies|     Agreement|BillingMethod|   PaymentMethod|MonthlyServiceCharges|TotalAmount|Churn|
+----------+------+-------------+-------+----------+------+-----------+-------------------+------------------+-------------------+-------------------+-----------------------+-------------------+-------------------+-------------------+--------------+-------------+----------------+---------------------+-----------+-----+
|2907-ILJBN|Female|          0.0|    

##### Q1 What is the average Monthly Charges for customers having “DSL” Internet connection. Consider senior citizen , male and churned customers whose tenure is greater than 60.  

In [28]:
First = data.filter((data['InternetConnection'] == 'DSL') & ((data['gender']== 'Male') & (data['SeniorCitizen']==1) & (data['Churn']=='Yes') & (data['tenure'] > 60))).select(mean('MonthlyServiceCharges')).show()

+--------------------------+
|avg(MonthlyServiceCharges)|
+--------------------------+
|         76.05276750166666|
+--------------------------+



##### Q2 What is the average amount for customers having “Mailed Check” as payment method and “One Year” as agreement. Consider the customers who have dependents and partner and have opted for call service.   

In [11]:
from pyspark.sql.functions import mean
second = data.filter((data['PaymentMethod'] == 'Mailed check') & (data['Agreement']=='One year') & (data['Dependents']=='Yes') & (data['Partner'] == 'Yes') & (data['CallService'] == 'Yes')).select(mean('TotalAmount')).show()

+------------------+
|  avg(TotalAmount)|
+------------------+
|1208.3856143694081|
+------------------+



##### Q3 What is the total Monthly Service Charges of customers  having different billing method. Consider the male and senior citizen customers whose tenure is less than 20 and have multiple connections  

In [31]:
from pyspark.sql.functions import sum
third = data.filter((data['BillingMethod']=='Yes') & (data['gender']== 'Male') & (data['SeniorCitizen']==1) & (data['tenure'] < 20) & (data['MultipleConnections']=='Yes')).select(sum('MonthlyServiceCharges')).show()

+--------------------------+
|sum(MonthlyServiceCharges)|
+--------------------------+
|             7906.29044558|
+--------------------------+



##### Q4 How many male and female customers has dependents and no dependents. Consider those customers who have monthly service charges greater than 100

In [22]:
from pyspark.sql.functions import count ,when,col
fourth = data.filter(data['MonthlyServiceCharges'] >100).groupBy("gender").agg(count(when(col('Dependents')=='Yes','True')), count(when(col('Dependents')=='No','True'))).show()

+------+-------------------------------------------------+------------------------------------------------+
|gender|count(CASE WHEN (Dependents = Yes) THEN True END)|count(CASE WHEN (Dependents = No) THEN True END)|
+------+-------------------------------------------------+------------------------------------------------+
|Female|                                              183|                                             438|
|  Male|                                              182|                                             389|
+------+-------------------------------------------------+------------------------------------------------+



##### Q5 How many number of customers have churned and not churned. Consider only female customers who have no dependents and has done call service and has preferred electronic check method. 

In [33]:
from pyspark.sql.functions import count
fifth = data.filter((data['gender']=='Female') & (data['Dependents']=='No') & (data['CallService']=='Yes') & (data['PaymentMethod']=='Electronic check')).agg(count(when(col('Churn')=='Yes','True')), count(when(col('Churn')=='No','True'))).show()


+--------------------------------------------+-------------------------------------------+
|count(CASE WHEN (Churn = Yes) THEN True END)|count(CASE WHEN (Churn = No) THEN True END)|
+--------------------------------------------+-------------------------------------------+
|                                         718|                                        566|
+--------------------------------------------+-------------------------------------------+



##### Q6 How many male and female customers have no dependents and have multiple connections. Consider the customers who have call service and has preferred either electronic check method or mailed check method

In [24]:
from pyspark.sql.functions import count
sixth = data.filter((data['CallService'] == 'Yes') & ((data['PaymentMethod'] == 'Electronic check') | (data['PaymentMethod'] == 'Mailed check'))).groupBy('gender').agg(count(when(col('Dependents')=='No','True')), count(when(col('MultipleConnections')=='Yes','True'))).show()

+------+------------------------------------------------+----------------------------------------------------------+
|gender|count(CASE WHEN (Dependents = No) THEN True END)|count(CASE WHEN (MultipleConnections = Yes) THEN True END)|
+------+------------------------------------------------+----------------------------------------------------------+
|Female|                                            2068|                                                      1252|
|  Male|                                            2071|                                                      1233|
+------+------------------------------------------------+----------------------------------------------------------+



##### Q8 What is the maximum monthly service charges of customers who have done payment by electronic check method? Consider only those customers who have agreement for on year or two years only. 

In [25]:
from pyspark.sql.functions import max
eight = data.filter((data['PaymentMethod'] == 'Electronic check') & ((data['Agreement'] == 'One year') | (data['Agreement'] == 'Two year'))).select(max('MonthlyServiceCharges')).show()

+--------------------------+
|max(MonthlyServiceCharges)|
+--------------------------+
|                    118.65|
+--------------------------+



##### Q9 What is the minimum total amount of male and female customers having one year or two year agreement type. Consider only those customers who have no internet connection, no online security no online backup and no device protection service.  

In [27]:
from pyspark.sql.functions import min
ninth = data.filter(((data['Agreement'] == 'One year') | (data['Agreement'] == 'Two year')) & ((data['InternetConnection'] == 'No') & (data['OnlineSecurity'] == 'No') & (data['OnlineBackup'] == 'No') & (data['DeviceProtectionService'] == 'No'))).groupBy('gender').agg(min('TotalAmount')).show()

+------+----------------+
|gender|min(TotalAmount)|
+------+----------------+
|Female|     69.21978888|
|  Male|     59.02463086|
+------+----------------+

